# HA EXTRACTEDTEXTS Q N TEM NENHUMA ALIAS, ANALISAR E ELIMINAR + CRIAR FILTRO BOOM PARA REMOVER REPETIDOS ?

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# if the url is the same, check the content to see if its repeated

because we used `&dedupValue=25&dedupField=url` and different aliases, we might have repeated data, so it's important to check for it

In [7]:
df = pd.read_parquet("data02.parquet")

def read_ExtractedText(linkToExtractedText):
    response = requests.get(linkToExtractedText)
    soup = BeautifulSoup(response.content, "html.parser")
    page_text = soup.get_text()
    return page_text

def remove_repeated_content(cell):
    """
    given one cell of the df, goes each result from the api request and find and remove duplicates
    ex.: remove_repeated_content(df.loc["Banco Comercial Português"]["api.2000"])
    """
    seen_links = {}
    for i in cell:
        if i["linkToArchive"] in seen_links:
            seen_links[i["linkToArchive"]] += 1
        else:
            seen_links[i["linkToArchive"]] = 1

    filtered_api = []
    repeated = {}
    for i in cell:
        if seen_links[i["linkToArchive"]] == 1:
            filtered_api.append(i)
        elif i["linkToArchive"] not in repeated:
                repeated[i["linkToArchive"]] = [read_ExtractedText(i["linkToExtractedText"])]
                filtered_api.append(i)
        else:
             text = read_ExtractedText(i["linkToExtractedText"])
             if text in repeated[i["linkToArchive"]]:
                  pass
             elif text not in repeated[i["linkToArchive"]]:
                  filtered_api.append(i)
                  repeated[i["linkToArchive"]].append(text)

    return filtered_api

df.loc[:, df.columns != 'aliases'] = df.loc[:, df.columns != 'aliases'].map(lambda x: remove_repeated_content(x))
df.to_parquet("data03.parquet")

In [8]:
df02 = pd.read_parquet("data02.parquet")#.set_index("companies")
df03 = pd.read_parquet("data03.parquet")#.set_index("companies")

df03.map(lambda x: len(x)) - df02.map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,0,-24,-21,-7,-12,-52,-66,-55
Galp Energia,0,-15,-8,-78,-24,-165,-78,-55
EDP,0,-3,-48,-15,-17,-1,-87,-9
Sonae,0,-2,0,0,-1,0,-61,-4
Mota-Engil,0,0,0,0,0,0,-31,0


In [9]:
pd.read_parquet("data03.parquet")["api.2015"].loc["Sonae"]

array([{'linkToArchive': 'https://arquivo.pt/wayback/20150227194532/http://www.publico.pt/economia/noticia/sonae-industria-vai-encerrar-actividade-industrial-em-franca-1687536', 'linkToExtractedText': 'https://arquivo.pt/textextracted?m=http%3A%2F%2Fwww.publico.pt%2Feconomia%2Fnoticia%2Fsonae-industria-vai-encerrar-actividade-industrial-em-franca-1687536%2F20150227194532', 'snippet': '<em>Sonae</em> Ind&uacute;stria sai do mercado franc&ecirc;s - P&Uacute;BLICO Artigos seguintes Artigos anteriores Mais <em>Sonae</em><span class="ellipsis"> ... </span> de Azevedo <em>SONAE</em> Empresas <em>Sonae</em> Ind&uacute;stria sai do mercado franc&ecirc;s 0 Partilhar no Facebook Partilhar no Twitter Partilhar no Google+ 0 <em>Sonae</em> Ind&uacute;stria sai do mercado franc&ecirc;s Por Rosa Soares 27/02/2015<span class="ellipsis"> ... </span>', 'title': 'Sonae Indústria sai do mercado francês - PÚBLICO', 'tstamp': '20150227194532'},
       {'linkToArchive': 'https://arquivo.pt/wayback/2015031519

# criar filtro bloom para quando for ler tudo, ver se n ha conteudos repetidos

# ....